In [2]:
import warnings
warnings.filterwarnings("ignore")
# import ST_utils
# import train_STAligner
import STAligner

# the location of R (used for the mclust clustering)
import os
os.environ['R_HOME'] = "/home/tguo/tguo2/miniconda3/envs/env_STAligner/lib/R"
os.environ['R_USER'] = "/home/tguo/tguo2/miniconda3/envs/env_STAligner/lib/python3.7/site-packages/rpy2"
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri


import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.sparse as sp
import scipy.linalg
from scipy import sparse

import torch
used_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
dirs="/data02/tguo/space_batch_effect/mouse_OB/"
Batch_list = []
adj_list = []
section_ids = ['Stereo_bin34','SlideV2']
extra_dirs="Stereo-bin34-filter/"
print(section_ids)
k=0
rad=[1.5,50]
for section_id in section_ids:
    print(section_id)
    count=pd.read_csv(dirs+"gtt_input/"+section_id+"_mat_allgene.csv",sep=',',header=0,index_col=0)
    meta = pd.DataFrame(pd.read_csv(dirs+"gtt_input/"+extra_dirs+section_id+"_meta.csv", sep=',', header=0, index_col=0).loc[:,'batch'])
    coord=pd.read_csv(dirs+"gtt_input/"+extra_dirs+section_id+"_coord.csv",header=0,index_col=0,sep=',')
    coord.columns=['x','y']
    if section_id=='SlideV2':
        used_barcodes=np.loadtxt(dirs+"origin/used_barcodes.txt",dtype=str)
        used_barcodes=['SlideV2-'+x for x in used_barcodes]
        cells=np.intersect1d(used_barcodes,count.index)
        count=count.loc[cells,:]
        meta=meta.loc[cells,:]
        coord=coord.loc[cells,:]
    count.index=meta.index
    adata = sc.AnnData(count)
    adata.X=sparse.csr_matrix(adata.X)
    adata.var_names_make_unique(join="++")
#     adata.obs['celltype'] = meta.loc[:,'celltype']
    adata.obs['batch'] = meta.loc[:,'batch']
    adata.obsm["spatial"] = coord.to_numpy()
    
    
    # Constructing the spatial network
    STAligner.Cal_Spatial_Net(adata, k_cutoff=8,model='Radius',rad_cutoff=rad[k]) # the spatial network are saved in adata.uns[‘adj’]
    # STAligner.Stats_Spatial_Net(adata) # plot the number of spatial neighbors
    
    # Normalization
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=5000)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata = adata[:, adata.var['highly_variable']]

    adj_list.append(adata.uns['adj'])
    Batch_list.append(adata)
    k+=1

['Stereo_bin34', 'SlideV2']
Stereo_bin34


In [13]:
adata_concat = ad.concat(Batch_list, label="slice_name", keys=section_ids)
adata_concat.obs['celltype'] = adata_concat.obs['celltype']
adata_concat.obs["batch_name"] = adata_concat.obs["batch"].astype('category')
print('adata_concat.shape: ', adata_concat.shape)
adj_concat = np.asarray(adj_list[0].todense())
for batch_id in range(1,len(section_ids)):
    adj_concat = scipy.linalg.block_diag(adj_concat, np.asarray(adj_list[batch_id].todense()))
adata_concat.uns['edgeList'] = np.nonzero(adj_concat)

adata_concat.shape:  (26963, 765)


In [14]:
%%time
adata_concat = STAligner.train_STAligner(adata_concat, verbose=True, knn_neigh = 100, device=used_device)

STAligner(
  (conv1): GATConv(765, 512, heads=1)
  (conv2): GATConv(512, 30, heads=1)
  (conv3): GATConv(30, 512, heads=1)
  (conv4): GATConv(512, 765, heads=1)
)
Pretrain with STAGATE...


100%|██████████| 500/500 [00:57<00:00,  8.77it/s]


Train with STAligner...


  0%|          | 0/500 [00:00<?, ?it/s]

Update spot triplets at epoch 500


...

Update spot triplets at epoch 800


 80%|████████  | 400/500 [01:18<00:11,  8.73it/s]

Update spot triplets at epoch 900


100%|██████████| 500/500 [01:39<00:00,  5.02it/s]


CPU times: user 3min 12s, sys: 31.3 s, total: 3min 43s
Wall time: 2min 41s


In [15]:
flags=''
for i in np.arange(len(section_ids)):
    flags=flags+'_'+section_ids[i]
cellnames=adata_concat.obs_names
embed=adata_concat.obsm['STAligner']
embednames=['STAligner-'+str(i) for i in np.arange(embed.shape[1])]
embed=pd.DataFrame(embed,index=cellnames,columns=embednames)
embed.to_csv(dirs+"methods/STAligner_mat"+flags+".txt")

In [ ]:
sc.pp.neighbors(adata_concat, use_rep='STAligner', random_state=666)
sc.tl.louvain(adata_concat, random_state=666, key_added="louvain", resolution=0.5)

sc.tl.umap(adata_concat, random_state=666)

section_color = ['#ff7f0e', '#1f77b4']
section_color_dict = dict(zip(section_ids, section_color))
adata_concat.uns['batch_name_colors'] = [section_color_dict[x] for x in adata_concat.obs.batch_name.cat.categories]

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams["figure.figsize"] = (3, 3)
plt.rcParams['font.size'] = 10

sc.pl.umap(adata_concat, color=['batch_name', 'louvain'], ncols=2, wspace=0.5, show=True)